In [1]:
import os
import time
import requests
from selenium import webdriver
import glob
import shutil

In [2]:
# How to execute this code
# Step 1 : pip install selenium. pillow, requests
# Step 2 : make sure you have chrome installed on your machine
# Step 3 : Check your chrome version ( go to three dot then help then about google chrome )
# Step 4 : Download the same chrome driver from here  " https://chromedriver.storage.googleapis.com/index.html "
# Step 5 : put it inside the same folder of this code

In [3]:
def fetch_image_urls(query: str, max_links_to_fetch: int, wd: webdriver, sleep_between_interactions: int = 0):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)

        # build the google query

    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)

        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")

        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [4]:
def persist_image(folder_path:str,url:str, counter):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        f = open(os.path.join(folder_path, 'jpg' + "_" + str(counter) + ".jpg"), 'wb')
        f.write(image_content)
        f.close()
        print(f"SUCCESS - saved {url} - as {folder_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [5]:
def search_and_download(search_term: str, driver_path: str, number_images: int, target_path='./train'):
    target_folder = os.path.join(target_path, '_'.join(search_term.lower().split(' '))) # make the folder name inside images with the search string

    if not os.path.exists(target_folder):
        os.makedirs(target_folder) # make directory using the target path if it doesn't exist already

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)

    counter = 0
    for elem in res:
        persist_image(target_folder, elem, counter)
        counter += 1

In [6]:
os.getcwd()

'C:\\Users\\egoeshu\\Desktop\\testingdoc\\imageScrapper'

In [7]:
# Changing webdriver path

os.chdir(r'C:\webdrivers')

In [8]:
import re

def move_filesto_test(number_images):
    test_folder= './test'

    for dir in os.listdir(r'./train'):
        files_list= []
        files_list= os.listdir(os.path.join('./train', dir)) #
        test_path= os.path.join(test_folder, dir)
        if not os.path.exists(test_path):
            os.makedirs(test_path) # make directory using the target path if it doesn't exist already
        for i in files_list:
            temp = re.findall(r'\d+', i)  # find digits in string
#           print(temp[0])
            if int(temp[0])>= (number_images-25): # sending 25 images to test sata
#             if int(temp[0])>= (number_images//3): # sending 95//3 images to test sata
#             if int(temp[0])>=11: # showing files having number equal or greater than 11
#               print(i)
                for jpgfile in glob.iglob(os.path.join(os.path.join('./train', dir), i)):
#                   shutil.copy(jpgfile, test_path)
                    shutil.move(jpgfile, test_path)
#                    print(jpgfile)
                

In [12]:
# driver = webdriver.Chrome()

DRIVER_PATH = './chromedriver'
# DRIVER_PATH= a
# search_term= ['BMW', 'Mercedes', 'Lamborgini', 'Tesla']

search_term = 'Tesla'
# num of images you can pass it from here  by default it's 10 if you are not passing
number_images = 95

search_and_download(search_term= search_term, driver_path=DRIVER_PATH, number_images= number_images)

# for keyword in search_term:
#     search_and_download(search_term=keyword, driver_path=DRIVER_PATH, number_images= number_images) # method to download images

move_filesto_test(number_images)

Found: 100 search results. Extracting links from 0:100
Found: 95 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTgHCT_n4_dtjbABcsvqC699cE44MRaLCLofA&usqp=CAU - as ./train\tesla
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQtlSdAIkKnpqygXaxWACvJ2HGOLi-nA937oQ&usqp=CAU - as ./train\tesla
SUCCESS - saved https://tesla-cdn.thron.com/delivery/public/image/tesla/c769509d-fe25-4915-9b6f-aeab622fa522/bvlatuR/std/0x0/screen1@2_1 - as ./train\tesla
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTH-G69nd0xA1A9H60Lfo1ObxFWI8hLic0A4g&usqp=CAU - as ./train\tesla
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTSMFNq4ZRnP_IhckxIPR6FvPAw5NZ21Bv1pA&usqp=CAU - as ./train\tesla
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRVHi5iwJXdrdz4kZf-R1w0HLYZ-vv_ZrvdBg&usqp=CAU - as ./train\tesla
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQif

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS-ov2-fsTK7bL1qhf6BaSVzoGP-h92iZMCVg&usqp=CAU - as ./train\tesla
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSpxYcX8rLLUbgB8u6ZLSqSIOaQ9U_N9GUyBw&usqp=CAU - as ./train\tesla
SUCCESS - saved https://cdn.cnn.com/cnnnext/dam/assets/191121212606-tesla-cybertruck-super-169.jpg - as ./train\tesla
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS0PTHtW3AH8UwEWtBiViWtUKcUB7yaXwuJqg&usqp=CAU - as ./train\tesla
SUCCESS - saved https://www.carscoops.com/wp-content/uploads/2020/08/Tesla-Drag-Race-1024x555.jpg - as ./train\tesla
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQSmuGng3cQ0qmApeIpgndP2q0qljJrU5br1Q&usqp=CAU - as ./train\tesla
SUCCESS - saved https://cnet1.cbsistatic.com/img/a17jR7KyAP4VzrCSIiZQoXjHl1A=/1200x675/2018/12/10/07e4f831-dcf0-4254-a1e6-4adf98b0846a/tesla-supercharging-ogi.jpg - as ./train\tesla
SUCCESS - saved https://www.cstatic

In [11]:
move_filesto_test(50)

In [69]:
import re

files_name= ['jpg 0 .jpg', 'jpg_15.jpg']

for i in files_name:
    temp = re.findall(r'\d+', i)
    if int(temp[0])>=10:
        print(temp[0])

15


In [67]:
a= ['1']
a[0]

'1'

In [ ]:
src_dir = "your/source/dir"
dst_dir = "your/destination/dir"
for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
    shutil.copy(jpgfile, dst_dir)

In [22]:
a=50
a//3

16